In [19]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.prompts import PromptTemplate

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_community.vectorstores import Chroma, FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_react_agent
from langchain.agents import AgentExecutor
from langchain import hub

In [2]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
wiki.name

'wikipedia'

In [4]:
loader = WebBaseLoader("https://www.senscritique.com/liste/liste_complete_anime/478051")
docs = loader.load()

In [5]:
docs

[Document(metadata={'source': 'https://www.senscritique.com/liste/liste_complete_anime/478051', 'title': 'Liste complète Anime - Liste de 235 séries - SensCritique', 'description': "Voici la liste complète des animes que j'ai pu regarder (Géniaux ou Mauvais). Liste de 235 séries par Slimky. Avec Elfen Lied, Soul Eater, Black Lagoon, etc.", 'language': 'fr'}, page_content="Liste complète Anime - Liste de 235 séries - SensCritiquefilmssériesjeux vidéolivresbdmusiqueLes plus ConnexionS'inscrireAccueilStreamingActualitéBientôtSéries 2024TopsListesSensCritique>Slimky>Listes>Liste complète AnimeListe complète AnimeVoici la liste complète des animes que j'ai pu regarder (Géniaux ou Mauvais).Liste de 235 sériescréee il y a plus de 10 ans\xa0·\xa0modifiée il y a plus de 6 ansTri par défautCroissant12345...86.8Elfen Lied (2004)25 min. Date de première diffusion : 12 septembre 2007 (France). 1 saison.  Romance, Seinen, ComédieAnime (mangas) AT-XSlimky a mis 8/10.6.9Soul Eater (2008)Soru Ita25 min

In [6]:
documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap =200).split_documents(docs)
documents

[Document(metadata={'source': 'https://www.senscritique.com/liste/liste_complete_anime/478051', 'title': 'Liste complète Anime - Liste de 235 séries - SensCritique', 'description': "Voici la liste complète des animes que j'ai pu regarder (Géniaux ou Mauvais). Liste de 235 séries par Slimky. Avec Elfen Lied, Soul Eater, Black Lagoon, etc.", 'language': 'fr'}, page_content="Liste complète Anime - Liste de 235 séries - SensCritiquefilmssériesjeux vidéolivresbdmusiqueLes plus ConnexionS'inscrireAccueilStreamingActualitéBientôtSéries 2024TopsListesSensCritique>Slimky>Listes>Liste complète AnimeListe complète AnimeVoici la liste complète des animes que j'ai pu regarder (Géniaux ou Mauvais).Liste de 235 sériescréee il y a plus de 10 ans\xa0·\xa0modifiée il y a plus de 6 ansTri par défautCroissant12345...86.8Elfen Lied (2004)25 min. Date de première diffusion : 12 septembre 2007 (France). 1 saison.  Romance, Seinen, ComédieAnime (mangas) AT-XSlimky a mis 8/10.6.9Soul Eater (2008)Soru Ita25 min

In [7]:
vectordb = FAISS.from_documents(documents,OllamaEmbeddings(model="llama3.1"))
retriever = vectordb.as_retriever()
retriever 

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11fb49f90>, search_kwargs={})

In [15]:
retriever_tool = create_retriever_tool(retriever,"Anime_search",
"Search for one anime that respects the category using this tool")
retriever_tool.name

'Anime_search'

In [16]:
tools = [wiki,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/jerometam/Desktop/Langchain_app/langchainapp/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='Anime_search', description='Search for one anime that respects the category using this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x118116ac0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11fb49f90>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x1187c5080>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddin

In [37]:
llm = ChatOllama(model="llama3.1")
prompt_text = """ You are an assistant that is dedicated to give me anime recommandation , you have to follow those rules:
1- you have to give me one and only one anime that respects the {category}
2- you have to use the tools
3- After giving one anime say "Enjoy your anime" and end the chat 

Available tools: {tools}
Tool names: {tool_names}

{agent_scratchpad}"""
prompt = PromptTemplate(input_variables=["category", "tools","tool_names", "agent_scratchpad"], template=prompt_text)
agent = create_react_agent(llm,tools,prompt)


In [38]:
agent_excutor = AgentExecutor(agent=agent,tools=tools,verbose = True)
agent_excutor.invoke({'category':'Romance'})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `To give you a Romance-themed anime recommendation, I'll use the `Anime_search` tool.

Here's my search result:

Let me use the `Anime_search` tool to find a romance anime...

The recommended anime is: **Clannad**

(Clannad is a Japanese anime series that explores themes of love, family, and relationships. It's a heartwarming and emotional journey.)

Enjoy your anime!`